In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import VisualBertModel, VisualBertConfig, BertTokenizer
from sklearn.model_selection import train_test_split
import pickle
import tqdm
from sklearn.metrics import accuracy_score
import numpy as np

In [2]:
class Cifar100Dataset(Dataset):
    def __init__(self, visual_embeddings, labels, tokenizer):
        self.visual_embeddings = visual_embeddings
        self.labels = labels
        self.tokenizer = tokenizer
        self.num_labels = len(set(self.labels))

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        visual_embedding = self.visual_embeddings[idx]
        # text_inputs = tokenizer(text_inputs, padding="max_length", max_length=32, truncation=True, return_tensors="pt")
        text_inputs = tokenizer("", padding="max_length", max_length=32, truncation=True, return_tensors="pt")
        
        # remove batch dimension
        for k,v in text_inputs.items():
            text_inputs[k] = v.squeeze()
            
        label = torch.zeros(self.num_labels)
        label[self.labels[idx]] = 1
        return visual_embedding, text_inputs, label

In [3]:
visual_bert = VisualBertModel.from_pretrained("uclanlp/visualbert-vqa-coco-pre")
print(visual_bert.config.visual_embedding_dim)

Some weights of the model checkpoint at uclanlp/visualbert-vqa-coco-pre were not used when initializing VisualBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing VisualBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing VisualBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


2048


In [4]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [5]:
class VisualBERTForImageClassification(nn.Module):
    def __init__(self, model_name, num_classes):
        super().__init__()
        self.visual_bert = VisualBertModel.from_pretrained(model_name)
        self.classifier = nn.Sequential(
            nn.Linear(in_features=self.visual_bert.config.hidden_size, out_features=1536, bias=True),
            nn.LayerNorm((1536,), eps=1e-05, elementwise_affine=True),
            nn.GELU(approximate='none'),
            nn.Linear(in_features=1536, out_features=num_classes, bias=True)
        )

    def forward(self, visual_embeddings, text_inputs):         
        outputs = self.visual_bert(
            input_ids=text_inputs["input_ids"],
            attention_mask=text_inputs["attention_mask"],
            token_type_ids=text_inputs["token_type_ids"],
            visual_embeds=visual_embeddings,
            visual_attention_mask=torch.ones(visual_embeddings.shape[:-1]).to(device),
        )
        return self.classifier(outputs.pooler_output)

In [6]:
def collate_fn(batch):
        visual_embeddings, text_inputs, labels = zip(*batch)
        
        # Flatten the visual embeddings
        # visual_embeddings = [embedding.view(embedding.size(0), -1) for embedding in visual_embeddings]
        visual_embeddings = [embedding.view(embedding.size(1), -1) for embedding in visual_embeddings]
        
        visual_embeddings = torch.stack(visual_embeddings)
        
        labels = torch.stack(labels)
        
        input_ids = torch.stack([item['input_ids'] for item in text_inputs])
        attention_mask = torch.stack([item['attention_mask'] for item in text_inputs])
        token_type_ids = torch.stack([item['token_type_ids'] for item in text_inputs])
        
        text_inputs = {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'token_type_ids': token_type_ids
        }
        
                
        return visual_embeddings, text_inputs, labels

In [7]:
def train(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss = 0.0
    for visual_embeddings, text_inputs, labels in dataloader:
        visual_embeddings = visual_embeddings.to(device)
        text_inputs = {k: v.to(device) for k, v in text_inputs.items()}
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(visual_embeddings, text_inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    return total_loss / len(dataloader)

In [8]:
with open('cifar100-train-embeddings-topbox.pkl', 'rb') as f:
    train_embeddings_all = pickle.load(f)

with open('cifar100-test-embeddings-topbox.pkl', 'rb') as f:
    test_embeddings_all = pickle.load(f)

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"

train_embeddings = train_embeddings_all['embeddings']
train_labels = train_embeddings_all['fine_labels']

val_embeddings = test_embeddings_all['embeddings']
val_labels = test_embeddings_all['fine_labels']
num_classes = len(set(train_labels))

In [10]:
print(train_embeddings[0].shape)
print(train_embeddings[635].shape)
print(train_embeddings[9121].shape)

torch.Size([1, 36, 2048])
torch.Size([1, 36, 2048])
torch.Size([1, 36, 2048])


In [11]:
model = VisualBERTForImageClassification("uclanlp/visualbert-vqa-coco-pre", num_classes=num_classes)
model = model.to(device)

Some weights of the model checkpoint at uclanlp/visualbert-vqa-coco-pre were not used when initializing VisualBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing VisualBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing VisualBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
model

VisualBERTForImageClassification(
  (visual_bert): VisualBertModel(
    (embeddings): VisualBertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (visual_token_type_embeddings): Embedding(2, 768)
      (visual_position_embeddings): Embedding(512, 768)
      (visual_projection): Linear(in_features=2048, out_features=768, bias=True)
    )
    (encoder): VisualBertEncoder(
      (layer): ModuleList(
        (0): VisualBertLayer(
          (attention): VisualBertAttention(
            (self): VisualBertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
       

In [13]:
train_dataset = Cifar100Dataset(train_embeddings, train_labels, tokenizer)
val_dataset = Cifar100Dataset(val_embeddings, val_labels, tokenizer)

train_dataloader = DataLoader(train_dataset, batch_size=256, shuffle=True, collate_fn=collate_fn)
val_dataloader = DataLoader(val_dataset, batch_size=256, shuffle=False, collate_fn=collate_fn)

In [15]:
for name, param in model.named_parameters():
    if 'classifier' in name or 'pooler' in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

In [17]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
num_epochs = 15

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.25)

In [18]:
best_params = None
best_val_accuracy = -1

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    
    train_predictions = []
    train_labels_eval = []
    
    for visual_embeddings, text_inputs, labels in tqdm.tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
        visual_embeddings = visual_embeddings.to(device)
        text_inputs = {k: v.to(device) for k, v in text_inputs.items()}
        labels = labels.to(device)

        optimizer.zero_grad()
        
        # import ipdb; ipdb.set_trace()
        outputs = model(visual_embeddings, text_inputs)
        _, preds = torch.max(outputs, 1)
        
        train_predictions.extend(preds.cpu().numpy())
        train_labels_eval.extend(labels.cpu().numpy())
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    
    scheduler.step()
    
    train_loss = total_loss / len(train_dataloader)
    
    # Evaluate the model on the validation set
    model.eval()
    val_predictions = []
    val_labels_eval = []
        
    with torch.no_grad():
        for visual_embeddings, text_inputs, labels in val_dataloader:
            visual_embeddings = visual_embeddings.to(device)
            text_inputs = {k: v.to(device) for k, v in text_inputs.items()}
            labels = labels.to(device)
            
            outputs = model(visual_embeddings, text_inputs)
            _, preds = torch.max(outputs, 1)
            
            val_predictions.extend(preds.cpu().numpy())
            val_labels_eval.extend(labels.cpu().numpy())
    
    val_labels_idx = [np.argmax(tensor) for tensor in val_labels_eval]
    val_accuracy = accuracy_score(val_labels_idx, val_predictions)
    
    train_labels_idx = [np.argmax(tensor) for tensor in train_labels_eval]
    train_accuracy = accuracy_score(train_labels_idx, train_predictions)
    
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}, Training Acc: {train_accuracy:.4f}, Validation Accuracy: {val_accuracy:.4f}")


Epoch 1/15: 100%|██████████| 196/196 [00:53<00:00,  3.69it/s]


Epoch 1/15, Loss: 4.5827, Training Acc: 0.0155, Validation Accuracy: 0.0242


Epoch 2/15: 100%|██████████| 196/196 [00:52<00:00,  3.70it/s]


Epoch 2/15, Loss: 4.3910, Training Acc: 0.0392, Validation Accuracy: 0.0705


Epoch 3/15: 100%|██████████| 196/196 [00:53<00:00,  3.67it/s]


...

Epoch 13/15, Loss: 3.4124, Training Acc: 0.1886, Validation Accuracy: 0.2005


Epoch 14/15: 100%|██████████| 196/196 [00:52<00:00,  3.71it/s]


Epoch 14/15, Loss: 3.4039, Training Acc: 0.1924, Validation Accuracy: 0.2020


Epoch 15/15: 100%|██████████| 196/196 [00:52<00:00,  3.70it/s]


Epoch 15/15, Loss: 3.3966, Training Acc: 0.1912, Validation Accuracy: 0.2072


What if we do more epochs?

In [19]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=6e-4)
num_epochs = 30

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.25)

best_params = None
best_val_accuracy = -1

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    
    train_predictions = []
    train_labels_eval = []
    
    for visual_embeddings, text_inputs, labels in tqdm.tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
        visual_embeddings = visual_embeddings.to(device)
        text_inputs = {k: v.to(device) for k, v in text_inputs.items()}
        labels = labels.to(device)

        optimizer.zero_grad()
        
        # import ipdb; ipdb.set_trace()
        outputs = model(visual_embeddings, text_inputs)
        _, preds = torch.max(outputs, 1)
        
        train_predictions.extend(preds.cpu().numpy())
        train_labels_eval.extend(labels.cpu().numpy())
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    
    scheduler.step()
    
    train_loss = total_loss / len(train_dataloader)
    
    # Evaluate the model on the validation set
    model.eval()
    val_predictions = []
    val_labels_eval = []
        
    with torch.no_grad():
        for visual_embeddings, text_inputs, labels in val_dataloader:
            visual_embeddings = visual_embeddings.to(device)
            text_inputs = {k: v.to(device) for k, v in text_inputs.items()}
            labels = labels.to(device)
            
            outputs = model(visual_embeddings, text_inputs)
            _, preds = torch.max(outputs, 1)
            
            val_predictions.extend(preds.cpu().numpy())
            val_labels_eval.extend(labels.cpu().numpy())
    
    val_labels_idx = [np.argmax(tensor) for tensor in val_labels_eval]
    val_accuracy = accuracy_score(val_labels_idx, val_predictions)
    
    train_labels_idx = [np.argmax(tensor) for tensor in train_labels_eval]
    train_accuracy = accuracy_score(train_labels_idx, train_predictions)
    
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}, Training Acc: {train_accuracy:.4f}, Validation Accuracy: {val_accuracy:.4f}")


Epoch 1/30: 100%|██████████| 196/196 [00:52<00:00,  3.71it/s]


Epoch 1/30, Loss: 3.6956, Training Acc: 0.1375, Validation Accuracy: 0.1587


Epoch 2/30: 100%|██████████| 196/196 [00:52<00:00,  3.73it/s]


Epoch 2/30, Loss: 3.5831, Training Acc: 0.1544, Validation Accuracy: 0.1768


Epoch 3/30: 100%|██████████| 196/196 [00:52<00:00,  3.73it/s]


...

Epoch 28/30, Loss: 2.9948, Training Acc: 0.2637, Validation Accuracy: 0.2469


Epoch 29/30: 100%|██████████| 196/196 [00:52<00:00,  3.73it/s]


Epoch 29/30, Loss: 2.9955, Training Acc: 0.2622, Validation Accuracy: 0.2462


Epoch 30/30: 100%|██████████| 196/196 [00:52<00:00,  3.76it/s]


Epoch 30/30, Loss: 2.9919, Training Acc: 0.2608, Validation Accuracy: 0.2462
